In [90]:
import sys
sys.path.append(r"C:/ami")
sys.path.append(r"C:/SyMAWoFlo/Symawofo")

import numpy as np
import json
from macro_utils import DotDict, MACROBOT_PATH
from ami.run.macrobot import MacrobotRunner

asset = 'FX_MXN_TR'
json_spec = (MACROBOT_PATH / f'data/preprocessing/specs/{asset}.json').read_text()
spec = DotDict(json.loads(json_spec))
spec.model.pop('loss')
spec.model.epochs = 3

In [91]:
spec.model.optimizer = 'rmsprop'

In [112]:
import tensorflow.keras.backend as kb
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow as tf
def custom_loss1(y_true, y_pred):
    k_abs_asymm = kb.switch(kb.less(y_true * y_pred, 0),
                             (2*kb.square(y_true - y_pred)),
                             (kb.square(y_true - y_pred)))
    loss = kb.sqrt(kb.mean(k_abs_asymm, axis=-1))
    return loss

def custom_loss2(y_true, y_pred):
        k_weights = kb.ones_like(y_true) * 0.01
        k_weights = kb.cumprod(k_weights)[::-1]
        k_weights = k_weights/kb.sum(k_weights)
        k_abs_asymm = kb.switch(kb.less(y_true * y_pred, 0),
                                 (2*kb.square(y_true - y_pred)),
                                 (kb.square(y_true - y_pred)))
        k_mul = keras.layers.multiply([k_abs_asymm, k_weights])
        loss = kb.sqrt(kb.sum(k_mul, axis=-1))
        return loss

In [122]:
from symawofo.models.tf import GRURegressor, CNNRegressor

x = np.random.randn(100, 44)
x2 = np.random.randn(100, 15, 44)
y = np.random.randn(100, 1)

# model = GRURegressor(**spec.model)
spec.model.epochs = 3
model = GRURegressor(loss=custom_loss2, **spec.model)

# sample_weight = 0.3**sample_weight
model.fit(x, y)






Epoch 1/3


1/1 [==============================] - 0s 2ms/step - loss: 0.2713
Epoch 2/3
1/1 [==============================] - 0s 5ms/step - loss: nan
Epoch 3/3
1/1 [==============================] - 0s 3ms/step - loss: nan


GRURegressor(
	model=None
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=<function custom_loss2 at 0x000002850ABE8430>
	metrics=None
	batch_size=1000
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=3
	type=GruModel
	use_bias=False
	nodes=12
	lookback_periods=15
	activation=selu
	kernel_initializer=random_uniform
	regularizer_l1=0.0035
	regularizer_l2=0.08
	recurrent_dropout=0.0
	dropout=0.0
	dense_units=()
	batch_norm=False
	_features_to_sample=None
	_index_to_sample=None
	_sample_weight_fn=None
)

In [123]:
model.model_.history.history

{'loss': [0.27130183577537537, nan, nan]}

In [121]:
spec.model.epochs = 0
model = GRURegressor(loss=custom_loss1, **spec.model)
model.fit(x, y)
sample_weight = np.ones(100)*0.1
sample_weight = np.cumprod(sample_weight)[::-1]
model.model_.compile(loss=custom_loss1)
model.model_.fit(x2, y, epochs=3, sample_weight=sample_weight, batch_size=1000)


Epoch 1/3


1/1 [==============================] - 0s 6ms/step - loss: 0.2503
Epoch 2/3
1/1 [==============================] - 0s 4ms/step - loss: 0.2148
Epoch 3/3
1/1 [==============================] - 0s 2ms/step - loss: 0.1911


In [120]:
model.model_.history.history

{'loss': [0.24612309038639069, 0.21093660593032837, 0.18775001168251038]}

In [87]:
model.model_.trainable_weights

[<tf.Variable 'gru_35/gru_cell_35/kernel:0' shape=(44, 36) dtype=float32, numpy=
 array([[ 0.0472813 , -0.01832223, -0.01339852, ..., -0.04555956,
         -0.01856278,  0.02134942],
        [ 0.00841048,  0.04764747, -0.00923449, ..., -0.03551576,
         -0.04871904, -0.0200743 ],
        [-0.0418419 ,  0.02125746, -0.03430023, ..., -0.01356485,
         -0.00590044, -0.00952414],
        ...,
        [ 0.00035453, -0.03919703, -0.04042126, ..., -0.00401983,
         -0.03879404, -0.0252171 ],
        [-0.00338913,  0.02198169,  0.04619229, ...,  0.04100705,
         -0.02026719,  0.02063413],
        [-0.03625999, -0.0120775 , -0.04247456, ...,  0.01836458,
          0.00971123,  0.02205313]], dtype=float32)>,
 <tf.Variable 'gru_35/gru_cell_35/recurrent_kernel:0' shape=(12, 36) dtype=float32, numpy=
 array([[ 7.38347769e-02, -1.28553227e-01,  2.93772798e-02,
         -4.06464607e-01, -2.23592035e-02, -1.18986778e-01,
          1.32262379e-01, -8.41015428e-02, -1.93071768e-01,
     

In [79]:
sample_weight = np.ones(100)*0.3
sample_weight = np.cumprod(sample_weight)[::-1]
model.model_.compile(loss=custom_loss1)
# model._build_keras_model() 
model.model_.fit(x2, y, sample_weight=sample_weight, batch_size=1000, epochs=0, random_state=0)

TypeError: fit() got an unexpected keyword argument 'random_state'

In [78]:
model.model_

In [3]:
spec.model

{'type': 'GruModel',
 'nodes': 12,
 'lookback_periods': 15,
 'dropout': 0.0,
 'regularizer_l1': 0.0035,
 'regularizer_l2': 0.08,
 'optimizer': 'adam',
 'activation': 'selu',
 'epochs': 10,
 'use_bias': False,
 'batch_norm': False,
 'batch_size': 1000}

In [ ]:
type(model)

In [ ]:
GRURegressor.mro()